In [ ]:
!git clone https://github.com/LFPy/ElectricBrainSignals.git
%cd ElectricBrainSignals/
!pip install -e

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.patches import Ellipse
import neuron
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

In [ ]:
def insert_synaptic_input(idx, cell, syn_scale):

    synapse_parameters = {'e': 0., # reversal potential
                          'weight': 0.002 * syn_scale, #  synapse weight
                          'record_current': True, # record synapse current
                          'syntype': 'Exp2Syn',
                          'tau1': 1, #Time constant, rise
                          'tau2': 3, #Time constant, decay
                          }
    synapse_parameters['idx'] = idx
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([1.]))
    return synapse, cell

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
tstop = 10
# Time window to extract spike from:
t0 = 3
t1 = 8

dt = 2**-6
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)
syn, cell = insert_synaptic_input(0, cell, 20)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

elec_radii = np.linspace(1, 500, num=10, dtype='int')

eaps = []
p2p_amps = []

elec_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = np.array([20]),
            y = np.array([0]),
            z = np.array([0]),
            method = 'root_as_point',
        )

for elec_radius in elec_radii:
    if elec_radius > 1e-9:
        elec_params.update(
            N = np.array([[1, 0, 0]]), # surface normals
            r = elec_radius,           # contact site radius
            n = elec_radius * 50,      # datapoints for averaging
        )

    elec = LFPy.RecExtElectrode(cell, **elec_params)
    LFP = elec.get_transformation_matrix() @ cell.imem * 1000
    eaps.append(LFP[0])
    p2p_amp = np.max(LFP[0]) - np.min(LFP[0])
    p2p_amps.append(p2p_amp)

In [ ]:
# create dotted line representing spike-detection threshold and get maximum amplitudes
max_amps = []
dots = []
for i in range(len(elec_radii)):
    max_amps.append(np.max(eaps[i]))
    dots.append(30)

In [ ]:
plt.plot(elec_radii, max_amps)
plt.plot(elec_radii, dots, linestyle='dotted')
plt.xlabel('Electrode radius, [µm]')
plt.title('Effect of electrode size on measured spike amplitudes')
plt.ylabel('Maximum amplitude, (µV)')

In [ ]:
# peak to peak amplitude plot
plt.semilogy(elec_radii, p2p_amps)
plt.semilogy(elec_radii, dots, linestyle='dotted', label='30 µV')
plt.xlabel('electrode radius (µm)')
plt.title('Effect of electrode size on measured spike amplitudes')
plt.ylabel(r"$V_{\rm e}$ amplitude (µV)")
plt.legend()

In [ ]:
# final figure of neuron morphology, electrodes, listed radii, membrane potential and ECP recordings

elec_clrs = {r: plt.cm.Reds(0.1 + i / (len(elec_radii) - 1))
             for i, r in enumerate(elec_radii)}

plt.close("all")
fig = plt.figure(figsize=[6, 3.2])
fig.subplots_adjust(left=0.14, bottom=0.18, top=0.8, right=0.98,
                    hspace=0.5)

ax_morph = fig.add_axes([0.01, 0.03, 0.7, 1.2], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[-350, 350],
                        ylim=[-600, 1200])

ax_eap = fig.add_axes([0.75, 0.7, 0.24, 0.2],
                      xlabel="time (ms)", ylabel="µV")
ax_eap_norm = fig.add_axes([0.75, 0.4, 0.24, 0.2],
                           xlabel="time (ms)", ylabel="normalized")
ax_p2p_amp = fig.add_axes([0.75, 0.1, 0.24, 0.2], 
                          xlabel='electrode radii (µm)', ylabel=r"$V_{\rm e}$ amplitude (µV)")
ax_mempot = fig.add_axes([0.75, 1, 0.24, 0.2],
                      xlabel="time (ms)", ylabel="mV")

zips = []
for x, z in cell.get_pt3d_polygons():
    zips.append(list(zip(x, z)))
polycol = PolyCollection(zips, edgecolors='none',
                         facecolors='gray', zorder=100, rasterized=False)
ax_morph.add_collection(polycol)

lines = []
line_names = []
for i, elec_radius in enumerate(elec_radii):
    c = elec_clrs[elec_radius]

    if elec_radius > 1e-9:
        el = Ellipse((elec.x[0], elec.z[0]), elec_radius / 2, 2*elec_radius,
                 facecolor=c, zorder=-i)
        ax_morph.add_artist(el)
    else:
        ax_morph.plot(elec.x[0], elec.z[0], '.', c=c, ms=3)

    l, = ax_eap.plot(cell.tvec, eaps[i], c=c)
    ax_eap_norm.plot(cell.tvec, eaps[i] / np.max(np.abs(eaps[i])), c=c)
    lines.append(l)
    line_names.append("$r$=%d µm" % elec_radius)
ax_p2p_amp.semilogy(elec_radii, p2p_amps)
ax_p2p_amp.semilogy(elec_radii, dots, linestyle='dotted', label='30 µV')
ax_mempot.plot(cell.tvec, cell.vmem[0])
ax_morph.legend(lines, line_names,bbox_to_anchor=(1, 0.52), loc='center left', borderaxespad=0) 
mark_subplots(ax_morph, ypos=1.02, xpos=0.1)
mark_subplots(ax_mempot, 'B', ypos=1.3)
mark_subplots([ax_eap, ax_eap_norm], "CD")
mark_subplots(ax_p2p_amp, 'E')
simplify_axes(fig.axes)
ax_p2p_amp.legend()
ax_p2p_amp.legend(bbox_to_anchor=(0.5, 0.68), prop={'size': 6})
# fig_folder = 'figure_one_kladd'
# fig_name = 'fig_1_electrode_size_effect_and_mem_pot'
#plt.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}.png', bbox_inches = "tight")